In [287]:
import torch
from torch import nn
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder

In [288]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(0)

In [289]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [290]:
print(df.info)

<bound method DataFrame.info of       customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL 

In [291]:
df.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [292]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [293]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna(subset=['TotalCharges'])

In [294]:
df['PaperlessBilling'].value_counts(normalize = False)

,count
PaperlessBilling,
Yes,4168
No,2864


In [295]:
df['gender'].value_counts(normalize = False) # 0 x

,count
gender,
Male,3549
Female,3483


In [296]:
df['InternetService'].value_counts(normalize=False) # 7

,count
InternetService,
Fiber optic,3096
DSL,2416
No,1520


In [297]:
df['MultipleLines'].value_counts(normalize=False) # 6

,count
MultipleLines,
No,3385
Yes,2967
No phone service,680


In [298]:
df['Contract'].value_counts(normalize = False)# 14

,count
Contract,
Month-to-month,3875
Two year,1685
One year,1472


In [299]:
df['PaymentMethod'].value_counts(normalize = False) # 16

,count
PaymentMethod,
Electronic check,2365
Mailed check,1604
Bank transfer (automatic),1542
Credit card (automatic),1521


In [300]:
df['TotalCharges'].isnull().sum()

np.int64(0)

In [301]:
X = df.iloc[:, 1:-1]
y = df.iloc[:, -1].values

In [302]:
le = LabelEncoder()
y = le.fit_transform(y)

# Splitting and Encoding the Data

In [303]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [304]:
# Encoding Categorical and Labeled data
categorical_data = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']
label_encoding_cols = ['gender', 'Partner', 'Dependents', 'PhoneService',
                'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), categorical_data),
                                     ('ordinal', OrdinalEncoder(), label_encoding_cols),
                                     ('num', StandardScaler(), numeric_cols)], remainder = "passthrough")

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [305]:
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)
X_train = torch.tensor(X_train, dtype = torch.float32)
X_test = torch.tensor(X_test, dtype = torch.float32)

In [306]:
X_train.shape

torch.Size([5625, 28])

# Models


In [307]:
class ChurnModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(28, 128)
    self.relu1 = nn.ReLU()

    self.layer_2 = nn.Linear(128, 128)
    self.relu2 = nn.ReLU()

    self.layer_3 = nn.Linear(128, 1)

  def forward(self, x):
    x = self.relu1(self.layer_1(x))
    x = self.relu2(self.layer_2(x))
    x = self.layer_3(x)
    return x

In [308]:
class ChurnModelV2(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(28, 128)
    self.relu1 = nn.ReLU()

    self.layer_2 = nn.Linear(128, 128)
    self.relu2 = nn.ReLU()

    self.layer_3 = nn.Linear(128, 1)

  def forward(self, x):
    x = self.relu1(self.layer_1(x))
    x = self.relu2(self.layer_2(x))
    x = self.layer_3(x)
    return x

In [309]:
class ChurnodelV3(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(28, 128)
    self.relu1 = nn.ReLU()

    self.layer_2 = nn.Linear(128, 128)
    self.relu2 = nn.ReLU()

    self.layer_3 = nn.Linear(128, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = self.relu1(self.layer_1(x))
    x = self.relu2(self.layer_2(x))
    x = self.layer_3(x)
    x = self.sigmoid(x)
    return x

In [319]:
model = ChurnModel()

In [311]:
with torch.inference_mode():
  y_pred = model(X_train)
  y_pred_probs = torch.sigmoid(y_pred)
  y_lables = torch.round(y_pred_probs)

print(f"Initial model accuracy : {accuracy_score(y_lables, y_train) * 100:.4f} %\n")
print(f"Initial Confusion matrix :\n {confusion_matrix(y_lables, y_train)}")

Initial model accuracy : 27.2178 %

Initial Confusion matrix :
 [[ 853  822]
 [3272  678]]


In [312]:
BCE_logit_loss = nn.BCEWithLogitsLoss()
BCE_loss = nn.BCELoss()
SGD_optim = torch.optim.SGD(params = model.parameters(), lr = 0.001)
Adam_Optim = torch.optim.Adam(params = model.parameters(), lr = 0.001)
RMSprop_optim = torch.optim.RMSprop(params = model.parameters(), lr = 0.001)

In [325]:
def model_loopV1(optimizer, loss_fn, epochs=150):

  epoch_counts = []
  test_loss_values = []
  train_loss_values = []
  test_accuracies = []

  for epoch in range(epochs):
    model.train()

    y_logits = model(X_train)
    y_pred_probs = torch.sigmoid(y_logits)
    y_lables = torch.round(y_pred_probs)

    loss = loss_fn(y_logits, y_train)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    model.eval()
    with torch.inference_mode():
      test_logits = model(X_test)
      test_pred_probs = torch.sigmoid(test_logits)
      test_labels = torch.round(test_pred_probs)

      test_loss = loss_fn(test_logits, y_test)

      test_accuracy = accuracy_score(test_labels, y_test)
      epoch_counts.append(epoch)
      test_loss_values.append(test_loss.item())
      train_loss_values.append(loss.item())
      test_accuracies.append(test_accuracy)
      if epoch % 10 == 0:
        print(f"Epochs : {epoch}  | Train Loss : {loss:.4f}  | Test Loss : {test_loss:.4f}  | Accuracy : {test_accuracy:.4f}")

In [314]:
def model_loopV2(optimizer, loss_fn, model, epochs = 150):
  epoch_counts = []
  test_loss_values = []
  train_loss_values = []
  test_accuracies = []

  for epoch in range(epochs):
    model.train()

    y_pred = model(X_train)

    loss = loss_fn(y_pred, y_train)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    model.eval()
    with torch.inference_mode():
      test_pred = model(X_test)

      test_loss = loss_fn(test_pred, y_test)

      test_accuracy = accuracy_score(test_pred, y_test)
      epoch_counts.append(epoch)
      test_loss_values.append(test_loss.item())
      train_loss_values.append(loss.item())
      test_accuracies.append(test_accuracy)
      if epoch % 10 == 0:
        print(f"Epochs : {epoch}  | Train Loss : {loss:.4f}  | Test Loss : {test_loss:.4f}  | Accuracy : {test_accuracy:.4f}")

In [326]:
model_loopV1(SGD_optim, BCE_logit_loss) # Stochastic Gradient Descent

Epochs : 0  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 10  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 20  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 30  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 40  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 50  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 60  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 70  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 80  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 90  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 100  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 110  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 120  | Train Loss : 0.6993  | Test Loss : 0.6986  | Ac

In [328]:
model_loopV1(Adam_Optim, BCE_logit_loss) # Adaptive moment estimator

Epochs : 0  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 10  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 20  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 30  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 40  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 50  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 60  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 70  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 80  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 90  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 100  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 110  | Train Loss : 0.6993  | Test Loss : 0.6986  | Accuracy : 0.3674
Epochs : 120  | Train Loss : 0.6993  | Test Loss : 0.6986  | Ac